# CSIRO - Image2Biomass with DINOv2 ViT
Fine-tunes Meta's DINOv2 Vision Transformer (via `timm`) on the CSIRO Image2Biomass dataset to predict five pasture biomass measurements from aerial imagery.

## Approach Overview
We load the provided train/test CSVs, reshape labels into per-image targets, and split folds by image. A DINOv2 backbone feeds a lightweight regression head, optimizing Smooth L1 losses across all biomass components. After training, we ensemble fold checkpoints to generate predictions and write the final `submission.csv`.

In [ ]:
!pip install -q timm==0.9.16 albumentations==1.4.14

In [ ]:
import osimport mathimport randomimport timefrom pathlib import Pathfrom dataclasses import dataclass, asdictimport numpy as npimport pandas as pdfrom PIL import Imageimport torchfrom torch import nnfrom torch.utils.data import Dataset, DataLoaderfrom torch.optim import AdamWfrom torch.optim.lr_scheduler import OneCycleLRfrom sklearn.model_selection import GroupKFoldimport timmfrom timm.data import create_transformprint(torch.__version__)

In [ ]:
@dataclassclass CFG:    data_dir: Path = Path('/kaggle/input/csiro-biomass')    output_dir: Path = Path('.')    seed: int = 2024    img_size: int = 392    batch_size: int = 16    num_workers: int = 4    epochs: int = 5    lr: float = 2e-4    min_lr: float = 1e-6    weight_decay: float = 1e-4    backbone: str = 'vit_base_patch14_dinov2'    hidden_dim: int = 512    drop_rate: float = 0.1    n_folds: int = 5    train_folds: tuple = (0,)    target_names: tuple = ('Dry_Clover_g','Dry_Dead_g','Dry_Green_g','Dry_Total_g','GDM_g')    num_targets: int = 5    use_amp: bool = True    grad_accum_steps: int = 1cfg = CFG()possible_dirs = [    Path('/kaggle/input/csiro-biomass'),    Path('/kaggle/input/CSIRO-I2B'),    Path('csiro-biomass'),]for p in possible_dirs:    if p.exists():        cfg.data_dir = p        breakprint(f"Using data from: {cfg.data_dir}")

In [ ]:
def seed_everything(seed: int = 42):    random.seed(seed)    np.random.seed(seed)    torch.manual_seed(seed)    torch.cuda.manual_seed_all(seed)    torch.backends.cudnn.deterministic = True    torch.backends.cudnn.benchmark = Falseseed_everything(cfg.seed)

In [ ]:
train_df = pd.read_csv(cfg.data_dir / 'train.csv')test_df = pd.read_csv(cfg.data_dir / 'test.csv')train_targets = train_df.pivot_table(    index='image_path',    columns='target_name',    values='target').reset_index().copy()train_targets.columns.name = Nonetrain_targets = train_targets[['image_path', *cfg.target_names]]train_targets = train_targets.fillna(0.0)train_targets['fold'] = -1kf = GroupKFold(n_splits=cfg.n_folds)for fold, (_, val_idx) in enumerate(kf.split(train_targets, groups=train_targets['image_path'])):    train_targets.loc[val_idx, 'fold'] = foldprint(train_targets.head())print(train_targets['fold'].value_counts())

In [ ]:
class BiomassDataset(Dataset):    def __init__(self, df: pd.DataFrame, mode: str, transform=None):        self.df = df.reset_index(drop=True)        self.mode = mode        self.transform = transform    def __len__(self):        return len(self.df)    def __getitem__(self, idx):        row = self.df.iloc[idx]        image_path = cfg.data_dir / row['image_path']        image = Image.open(image_path).convert('RGB')        if self.transform:            image = self.transform(image)        else:            image = transforms.ToTensor()(image)        sample = {            'pixel_values': image,            'image_path': row['image_path']        }        if self.mode != 'test':            target = torch.tensor(row[cfg.target_names].values, dtype=torch.float32)            sample['targets'] = target        return sample

In [ ]:
from torchvision import transformsdef get_transforms(is_train=True):    if is_train:        return create_transform(            input_size=(3, cfg.img_size, cfg.img_size),            is_training=True,            auto_augment='rand-m9-mstd0.5-inc1',            interpolation='bicubic',            re_prob=0.25,            re_mode='pixel',            re_count=1,        )    else:        return create_transform(            input_size=(3, cfg.img_size, cfg.img_size),            is_training=False,            interpolation='bicubic',        )

In [ ]:
class DinoRegressor(nn.Module):    def __init__(self, cfg):        super().__init__()        self.backbone = timm.create_model(            cfg.backbone,            pretrained=True,            num_classes=0,            global_pool='avg'        )        in_features = self.backbone.num_features        self.head = nn.Sequential(            nn.LayerNorm(in_features),            nn.Dropout(cfg.drop_rate),            nn.Linear(in_features, cfg.hidden_dim),            nn.GELU(),            nn.Dropout(cfg.drop_rate),            nn.Linear(cfg.hidden_dim, cfg.num_targets)        )    def forward(self, pixel_values):        feats = self.backbone(pixel_values)        return self.head(feats)

In [ ]:
class AverageMeter:    def __init__(self):        self.reset()    def reset(self):        self.val = 0        self.avg = 0        self.sum = 0        self.count = 0    def update(self, val, n=1):        self.val = val        self.sum += val * n        self.count += n        self.avg = self.sum / self.count if self.count != 0 else 0def get_dataloaders(train_df, valid_df):    train_dataset = BiomassDataset(train_df, mode='train', transform=get_transforms(True))    valid_dataset = BiomassDataset(valid_df, mode='valid', transform=get_transforms(False))    train_loader = DataLoader(        train_dataset,        batch_size=cfg.batch_size,        shuffle=True,        num_workers=cfg.num_workers,        pin_memory=True,        drop_last=True,    )    valid_loader = DataLoader(        valid_dataset,        batch_size=cfg.batch_size,        shuffle=False,        num_workers=cfg.num_workers,        pin_memory=True,        drop_last=False,    )    return train_loader, valid_loader

In [ ]:
def train_one_epoch(model, loader, criterion, optimizer, scheduler, scaler, device):    model.train()    loss_meter = AverageMeter()    for step, batch in enumerate(loader):        images = batch['pixel_values'].to(device, non_blocking=True)        targets = batch['targets'].to(device, non_blocking=True)        optimizer.zero_grad()        with torch.cuda.amp.autocast(enabled=cfg.use_amp):            preds = model(images)            loss = criterion(preds, targets)        scaler.scale(loss).backward()        scaler.step(optimizer)        scaler.update()        if scheduler is not None:            scheduler.step()        loss_meter.update(loss.item(), images.size(0))    return loss_meter.avgdef validate_one_epoch(model, loader, criterion, device):    model.eval()    loss_meter = AverageMeter()    preds_list = []    with torch.no_grad():        for batch in loader:            images = batch['pixel_values'].to(device, non_blocking=True)            targets = batch['targets'].to(device, non_blocking=True)            with torch.cuda.amp.autocast(enabled=cfg.use_amp):                preds = model(images)                loss = criterion(preds, targets)            loss_meter.update(loss.item(), images.size(0))            preds_list.append(preds.detach().cpu())    predictions = torch.cat(preds_list).numpy()    return loss_meter.avg, predictions

In [ ]:
DEVICE = 'cuda' if torch.cuda.is_available() else 'cpu'criterion = nn.SmoothL1Loss()oof_predictions = np.zeros((len(train_targets), cfg.num_targets), dtype=np.float32)test_images = pd.DataFrame({'image_path': test_df['image_path'].unique()})all_fold_models = []for fold in range(cfg.n_folds):    if fold not in cfg.train_folds:        continue    print(f"===== Fold {fold} =====")    train_split = train_targets[train_targets['fold'] != fold].reset_index(drop=True)    valid_split = train_targets[train_targets['fold'] == fold].reset_index(drop=True)    train_loader, valid_loader = get_dataloaders(train_split, valid_split)    model = DinoRegressor(cfg).to(DEVICE)    optimizer = AdamW(model.parameters(), lr=cfg.lr, weight_decay=cfg.weight_decay)    steps_per_epoch = len(train_loader)    scheduler = OneCycleLR(        optimizer,        max_lr=cfg.lr,        epochs=cfg.epochs,        steps_per_epoch=steps_per_epoch,        pct_start=0.1,        div_factor=25,        final_div_factor=cfg.lr / cfg.min_lr,    )    scaler = torch.cuda.amp.GradScaler(enabled=cfg.use_amp)    best_loss = float('inf')    best_state = None    for epoch in range(cfg.epochs):        start = time.time()        train_loss = train_one_epoch(model, train_loader, criterion, optimizer, scheduler, scaler, DEVICE)        val_loss, val_preds = validate_one_epoch(model, valid_loader, criterion, DEVICE)        duration = time.time() - start        print(f"Epoch {epoch+1}/{cfg.epochs} | train: {train_loss:.4f} | valid: {val_loss:.4f} | {duration:.1f}s")        if val_loss < best_loss:            best_loss = val_loss            best_state = model.state_dict()    model.load_state_dict(best_state)    all_fold_models.append(best_state)    fold_idx = valid_split.index    oof_predictions[fold_idx] = val_predsnp.save(cfg.output_dir / 'oof_predictions.npy', oof_predictions)

In [ ]:
@torch.no_grad()
def infer(model_states, df):
    models = []
    for state in model_states:
        model = DinoRegressor(cfg).to(DEVICE)
        model.load_state_dict(state, strict=True)
        model.eval()
        models.append(model)
    dataset = BiomassDataset(df, mode='test', transform=get_transforms(False))
    loader = DataLoader(
        dataset,
        batch_size=cfg.batch_size,
        shuffle=False,
        num_workers=cfg.num_workers,
        pin_memory=True,
    )
    image_preds = {}
    for batch in loader:
        images = batch['pixel_values'].to(DEVICE, non_blocking=True)
        preds = torch.zeros((images.size(0), cfg.num_targets), device=DEVICE)
        for model in models:
            preds += model(images)
        preds /= len(models)
        preds = preds.detach().cpu().numpy()
        for path, pred in zip(batch['image_path'], preds):
            image_preds[path] = pred
    return image_preds

ensemble_preds = infer(all_fold_models, test_images)


In [ ]:
submission = test_df.copy()submission['target'] = submission.apply(    lambda row: ensemble_preds[row['image_path']][cfg.target_names.index(row['target_name'])],    axis=1)submission[['sample_id', 'target']].to_csv('submission.csv', index=False)submission.head()